### Code for the gel locater

In [1]:
#loading in packages
import numpy as np
import cv2
from skimage import measure
from skimage.io import imread
import matplotlib.pyplot as plt

In [3]:
#creating a function to locate the boxes
def locate_boxes(mask_image, method='center'): #input is the mask image as a numpy array or file path (depending on how we want to load in the data), we can change this
    if isinstance(mask_image, str): #this if statement is loading the image if its a file path
        mask = imread(mask_image, as_gray=True)
        mask = (mask > 0).astype(np.uint8) #converts the mask
    else: #this else statement loads the image if its a numpy array
        mask = mask_image.astype(np.uint8)
    labels = measure.label(mask, connectivity=2) #labels components that are connected
    props = measure.regionprops(labels)
    return labels, props
    
def get_box_locations(props, method='center'): #getting the box locations
    box_locations = []
    for prop in props:
        if method == 'center': #converts centroit (y,x) to (x,y)
            y, x = prop.centroid
            box_locations.append((int(x), int(y)))
        elif method == 'topleft':# converts centroid (x, y) with x=col, y=row
            min_row, min_col, _, _ = prop.bbox 
            box_locations.append((min_col, min_row))
        elif method == 'bottomleft': #converts cetroid (x, y) with x=col, y=row
            _, min_col, max_row, _ = prop.bbox
            box_locations.append((min_col, max_row - 1))
    
    return box_locations
    

def visualize_boxes(mask_image, box_locations, method='center'): #visualizing the masked image with detected boxes
    if isinstance(mask_image, str):
        mask = imread(mask_image, as_gray=True)
        # Ensure binary
        mask = (mask > 0).astype(np.uint8)
    else:
        mask = mask_image.astype(np.uint8)
    vis_img = np.stack([mask*255, mask*255, mask*255], axis=-1) #converting to create an image
    index = 1  
    for box in box_locations:
        x, y = box  
        cv2.circle(vis_img, (x, y), 3, (255, 0, 0), -1) #red dot
        cv2.putText(vis_img, f"Box {index}: ({x},{y})", (x + 5, y),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
        index += 1
    plt.figure(figsize=(10, 10))
    plt.imshow(vis_img)
    plt.title(f'Box Locations using {method} method')
    plt.axis('on')
    plt.grid(True)
    plt.show()
